<a href="https://colab.research.google.com/github/Byeon-MJ/Dacon_Repo/blob/main/Dacon_Wallpaper_Classification/Dacon_Wallpaper_Classification_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch + EfficientNet B0을 활용한 이미지 분류모델(Baseline)

## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler